#Web scraping del sito della collana di libri Adelphi 
Una delle task più comuni per cui viene impiegato Python è il così detto **web scraping**. Il web scraping consiste nell'automatizzare l'ottenimento di alcune informazioni dal web tramite un codice e di utilizzarle di conseguenza. La qualità e la semplicità nell'ottenimento delle informazioni è dato solitamente dalla struttura della fonte: alcuni siti sono molto più semplici da navigare mentre altri possono essere più complessi. Per questo motivo il web scraping è spesso molto complesso. 
In questo esercizio vi verrà fornito uno scraper già fatto in grado di ottenere informazioni dal catalogo online della nota collana di **libri Adelphi** (https://www.adelphi.it/): calandovi nei panni di un avido lettore e collezionista di libri, il vostro obiettivo sarà di utilizzarlo per digitalizzare interamente la vostra collezione di libri, in modo da poter velocemente controllare l'inventario dei vostri libri ed alcune informazioni a riguardo con un semplice comando ogni volta che lo vorrete.

In [1]:
# IMPORTANTE
# questa cella di codice serve ad installare nel vostro Python due tra i principali strumenti di web scraping: beautiful soup e requests.
# qualora di fossero problemi con la loro installazione o import non esitate a contattarci
!pip install requests
!pip install beautifulsoup4

# Implementazione dello scraper
Come anticipato, non vi sarà richiesto di implementare lo scraper. Tuttavia gli interessati possono trovare informazioni su come utilizzare al meglio le due librerie per lo scraping *requests* e *beautifulsoup* e cercare di comprendere il codice della classe AdelphiScraper. Se non siete tra questi potete saltare alla prossima sezione.
Qui qualche risorsa:

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

https://requests.readthedocs.io/en/master/user/quickstart/ 

In [2]:
import requests as req
import bs4 as bs

In [3]:
class AdelphiScraper():
  def __init__(self):
    self.catalog      = 'https://www.adelphi.it/catalogo/'
    self.subject      = 'https://www.adelphi.it/catalogo/materia/'
    self.subjects_map = self.init_subjects()

  def init_subjects(self):
    resp  = req.get(self.catalog)
    soup  = bs.BeautifulSoup(resp.text, "html.parser")
    items = soup.find_all('ul', class_='subject')
    ret = dict()
    for i in items:
      for j in i:
        ret.update({int(j.a['href'].split('/')[-1]) : j.text})
    return ret

  def get_subject_list(self):
    return self.subjects_map
  
  def get_subject_books(self, subject_code):
    page_found = True
    suffix = ''
    page_counter = 1
    books = dict()
    isbns = dict()
    while page_found:
      try:
        resp = req.get(self.subject + str(subject_code) + suffix, allow_redirects = False)
        soup = bs.BeautifulSoup(resp.text, "html.parser")
        if not len(soup):
          raise ValueError()
        book_infos, isbn_maps = self.extract_books(soup)
        books.update(book_infos)
        isbns.update(isbn_maps)
        page_counter += 1
        suffix = '/p' + str(page_counter)
      except:
        page_found = False
    return books, isbns

  def extract_books(self, soup):
    items = soup.find_all('div', class_='list_impressum')
    items2 = soup.find_all('div', class_='data' )
    items3 = soup.find_all('div', class_ = "abstract hidden-xs")
    isbn_map  = dict()
    book_info = dict()
    counter = 0
    for i in items:
      isbn_map.update({i.a['href'].split('/')[-1] : i.a.text}) #isbn : titolo
      book_info.update({i.a.text : {'Autore': i.div.text, \
                                    'Anno': items2[counter].text.split(' ')[0].split(',')[0], \
                                    'Prezzo Originale' : i.span.text.split(' ')[1].replace(',','.'), \
                                    'Sconto': i.span.text.split(' ')[2], \
                                    'Pagine' :items2[counter].text.split(' ')[2].split(',')[0].split('-')[-1], \
                                    'Abstract': items3[counter].text}})
      counter += 1
    return book_info, isbn_map

#Utilizzo della classe AdelphiScraper

*   La classe AdelphiScraper non richiede alcun argomento di inizializzazione.
*   Essendo un WebScraper richiede ovviamente la connessione ad internet.
*   Essa contiene al suo interno diversi attributi e metodi, i metodi che vi serviranno sono:
    * `get_subject_list()` : questo metodo restituisce un dizionario contenente tutte le categorie della collana di libri Adelphi e il codice numerico a cui sono associate sul sito.
    * `get_subject_books(category_code)` : questo metodo riceve il codice numerico (**come intero**) di una categoria e restituisce due dizionari: 
        * Il primo ha come chiave i titoli dei libri appartenenti alla categoria e come valori svariati dati ad essi inerenti (Autore, prezzo, anno di stampa dell'ultima edizione, etc.)
        * Il secondo ha come chiavi gli ISBN (codice univoco diverso da libro a libro) e come valori i titoli dei libri associati.




In [4]:
## Come prima cosa istanziate un oggetto di classe AdelphiScraper e provate ad utilizzare i due metodi sopra descritti per osservarne l'output,
## questa operazione è molto importante al fine di capire bene quali sono le strutture dati a vostra dispozione.
## ATTENZIONE : osservate bene quali sono i tipi dei dati contenuti nelle strutture che riceverete in output, più avanti avrete bisogno di farvi operazioni o conversioni.

Ad_Scraper = AdelphiScraper()
print(Ad_Scraper.get_subject_list())
print((Ad_Scraper.get_subject_books(28))[1])


test_dict=dict()
test_dict[28]=Ad_Scraper.get_subject_books(28)
test_dict[59]=(Ad_Scraper.get_subject_books(59))
test_dict[80]=(Ad_Scraper.get_subject_books(80))
test_dict[88]=(Ad_Scraper.get_subject_books(88))

print(test_dict[28][1])





{54: 'Aforismi e frammenti', 49: 'Antichità classica', 30: 'Antropologia', 71: 'Architettura', 26: 'Arte', 3: 'Autobiografie', 25: 'Biografie', 50: 'Biologia', 82: 'Buddhismo', 63: 'Cina', 12: 'Cinema', 70: 'Civiltà arcaiche', 15: 'Classici greci e latini', 88: 'Cosmologia', 11: 'Cristianesimo', 19: 'Critica della cultura', 18: 'Critica e storia letteraria', 87: 'Cucina', 53: 'Diari', 85: 'Diritto', 45: 'Ebraismo e letteratura ebraica', 9: 'Economia', 76: 'Editoria', 90: 'Egittologia', 4: 'Epistolari', 44: 'Eros', 60: 'Etologia', 75: 'Fantascienza', 33: 'Filosofia', 62: 'Filosofia della scienza', 28: 'Fisica', 55: 'Fotografia', 66: 'Giappone', 42: 'Giardinaggio', 67: 'Giochi', 6: 'India', 17: 'Interviste', 68: 'Islam', 93: 'Letteratura coreana', 16: 'Letteratura di viaggi', 91: 'Letteratura ellenica', 84: 'Letteratura fantastica', 21: 'Letteratura francese', 1: 'Letteratura inglese', 8: 'Letteratura italiana', 47: 'Letteratura mitteleuropea', 74: 'Letteratura neerlandese', 13: 'Lettera

# Creazione di una libreria digitale
Richieste dell'esercizio:

*   Implementare la classe `Book(ISBN)`: questa classe dovrà essere inizializzata soltanto dall'ISBN di un libro, utilizzando AdelphiScraper dovrà essere in grado di inizializzarsi con i seguenti attributi:
    *   `self.ISBN` : ISBN del libro.
    *   `self.title` : il titolo del libro.
    *   `self.print_year` : l'anno dell'ultima ristampa.
    *   `self.author` : l'autore del libro.
    *   `self.abstract` : abstract del libro.
    *   `self.pages` : numero di pagine del libro.
    *   `self.category` : categoria a cui appartiene il libro.

  **Suggerimento**: tutte queste informazioni devono essere reperite con varie chiamate ai metodi di AdelphiScraper e tramite trasformazioni alle strutture dati ricevute, si consiglia caldamente di chiamare AdelphiScraper esternamente e creare una struttura dati adeguata a cercare tutte le informazioni velocemente dato **soltanto** l'ISBN di un libro, a quel punto la classe Book potrà attingere da lì senza connettersi allo scraper in ogni sua istanza.

  Dovrà inoltre possedere i seguenti metodi:
    *   `get_whole_price(self)` : restituirà il prezzo lordo del libro usando AdelphiScraper.
    *   `get_net_price(self)` : restituirà il prezzo netto del libro usando AdelphiScraper.
    *   `print_book_infos(self)` : stamperà in output le varie informazioni sul libro.
    *   `abstract_len` : restituisce lunghezza dell'abstract del libro

* Vi è richiesto di digitalizzare le informazioni di alcuni libri, creare una classe chiamata `MyLibrary(ISBN_list)` che possa essere inizializzata soltanto con una lista di ISBN. In modo simile alla classe `Book(ISBN)` questa dovrà far uso di AdelphiScraper per collezionare le informazioni di tutti i libri in essa contenuti, si suggerisce l'utilizzo di dizionari e oggetti di tipo Book come strutture dati replicando gli stessi attributi e metodi (in questi ultimi si possa specificare il libro del quale siamo interessati al prezzo, all'abstract etc.).
La classe dovrà inoltre avere i seguenti metodi:
    * `insert_book(self, ISBN) `: inserisca nella libreria l'oggetto di tipo Book corrispondente all'ISBN passato.
    * `remove_book(self, ISBN) `: rimuova dall libreria l'oggetto di tipo Book corrispondente all'ISBN passato.
    * `get_library_value(self) `: restituirà il valore totale (dato dai prezzi) dei libri contenuti nella libreria, sia lordo che al netto degli sconti.
    * `get_total_pages(self) ` : restituirà il numero totale di pagine dei libri nella nostra libreria
    * `get_categories_number(self) ` : restituirà il numero di categorie differenti che sono presenti nella nostra libreria.

* Introduciamo il concetto di libri "vicini": diciamo che due libri sono vicini se appartengono alla stessa categoria **oppure** il loro ISBN modulo 3 è lo stesso: se ad esempio due libri hanno come ISBN 134434**6** e ISBN 294823**9** questi avranno entrambi ISBN modulo 3 uguale a 0 e saranno vicini.
    * Si inserisca nella classe `Book(ISBN)` il metodo `is_near(self, book_2)` che restituisca `True` se l'oggetto di tipo Book passato in input è vicino al libro rappresentato dall'oggetto Book che chiama il metodo, `False` altrimenti.
```
b1 = Book(123456)
b2 = Book(123456789)
b1.is_near(b2)
>> True
```
    * Si inserisca nella classe `MyLibrary(ISBN_list)` il metodo `minimum_path(self, start_ISBN, end_ISBN) ` che riceve in input due ISBN appartenenti alla libreria, uno iniziale ed uno finale, e restituisca la lista ordinata degli ISBN dei libri facenti parte della libreria da cui bisogna passare per arrivare da quello iniziale a quello finale. Tuttavia, è possibile passare da un libro ad un altro solo se sono vicini, questa funzione deve restituire la sequenza di passaggi di lunghezza **più breve** per arrivare a destinazione.
    * Si inserisca nella classe `MyLibrary(ISBN_list)` il metodo `minimum_cost_path(self, start_ISBN, end_ISBN) `, una versione alternativa del metodo precedente che invece che il cammino di lunghezza minore restituisce quello con il costo **netto** totale dei libri facenti parte del cammino minore possibile.

Si usino i seguenti ISBN come caso prova delle varie implementazioni:

Cose che potrebbero esservi utili: 
  * Attributi dei dizionari: .keys(), .values(), ...
  * Teoria dei grafi
  

In [5]:
class Book:
    def __init__(self, ISBN, test_dict): #metto anche test_dict per eseguire i test(se guardi le funzioni get_title e get_book sono senza test_dict ma fanno riferimento allo scraper originale)
        self.ISBN=ISBN
        self.title=(self.test_get_title(self.ISBN, test_dict))[0] #uso test_get_title per eseguire test
        self.category=(self.test_get_title(self.ISBN, test_dict))[1]
        self.book=self.test_get_book(self.title, test_dict) #uso test_get_book per eseguire test
        self.print_year=(self.book)['Anno']
        self.author=(self.book)['Autore']
        self.abstract=(self.book)['Abstract']
        self.pages=(self.book)['Pagine']
        
    
    def test_get_title(self, ISBN, test_dict): #funzione di test
        found=False
        for cat in test_dict:
            for el in (test_dict[cat])[1]:
                if el==ISBN:
                    found=True
                    return [((test_dict[cat])[1])[el], cat]
        if found==False:
            print("Libro non trovato")

        
    def get_title(self, ISBN):
        found=False
        Dict_of_categories=Ad_Scraper.get_subject_list()
        for cat in Dict_of_categories:
            ISBN_dictionary=(Ad_Scraper.get_subject_books( Dict_of_categories[cat]))[1]
            for el in ISBN_dictionary:
                if el==ISBN:
                    found=True
                    return [ISBN_dictionary[el], cat]
        if found==False:
            print("Libro non trovato")

    def test_get_book(self, title, test_dict): #funzione di test
            found=False
            for cat in test_dict:
                for book in (test_dict[cat])[0]:
                    if book==title:
                         return ((test_dict[cat])[0])[title]
    
    def get_book(self, title):
            found=False
            Dict_of_categories=Ad_Scraper.get_subject_list()
            for cat in Dict_of_categories:
                book_dictionary=(Ad_Scraper.get_subject_books( Dict_of_categories[cat]))[0]
                for book in book_dictionary:
                    if book==title:
                         return book_dictionary[title]

    def get_whole_price(self):
        return float(self.book['Prezzo Originale'])
        
    def get_net_price(self):
        discount=self.book['Sconto']
        discount=discount.replace("-", "")
        discount=discount.replace("%", "")
        return self.get_whole_price()*(1-float(discount)/100)
        
    def print_book_infos(self):
        print(self.book)
            
    def abstract_len(self):
        return len(self.abstract)
        
    def get_pages(self):
        return int(self.pages)
        
    def is_near(self, book_2):
        if self.category==Book(book_2).category:
            return True
        if (int((self.ISBN))%3)==(int((book_2))%3):
            return True
        return False
    
    def test_is_near(self, book_2, test_dict): #funzione di test
        if self.category==Book(book_2, test_dict).category:
            return True
        if (int((self.ISBN))%3)==(int((book_2))%3):
            return True
        return False
    
    
    
print(Book('9788845908736', test_dict).title)
print(Book('9788845908736', test_dict).category)
print(Book('9788845908736', test_dict).print_year)
print(Book('9788845908736', test_dict).author)
print(Book('9788845908736', test_dict).abstract)
print(Book('9788845908736', test_dict).pages)
print(Book('9788845908736', test_dict).get_whole_price())
print(Book('9788845908736', test_dict).get_net_price())
Book('9788845908736', test_dict).print_book_infos()
print(Book('9788845908736', test_dict).abstract_len())
print(Book('9788845908736', test_dict).test_is_near('9788845934513', test_dict))


    


Il mondo dentro il mondo
28
2002
John D. Barrow
Davvero esistono “lì fuori” leggi di natura, che stanno in attesa di essere scoperte, indipendenti dal nostro modo di pensare, o esse rappresentano soltanto la descrizione più conveniente di ciò che abbiamo visto? Come è nata l’idea stessa di “leggi di natura”? Queste costituiscono la realtà profonda o sono soltanto pezzi di un regolamento...
770
40.0
38.0
{'Autore': 'John D. Barrow', 'Anno': '2002', 'Prezzo Originale': '40.00', 'Sconto': '-5%', 'Pagine': '770', 'Abstract': 'Davvero esistono “lì fuori” leggi di natura, che stanno in attesa di essere scoperte, indipendenti dal nostro modo di pensare, o esse rappresentano soltanto la descrizione più conveniente di ciò che abbiamo visto? Come è nata l’idea stessa di “leggi di natura”? Queste costituiscono la realtà profonda o sono soltanto pezzi di un regolamento...'}
344
True


In [15]:
import sys 
  
sys.setrecursionlimit(3000) 


class My_Library:
    def __init__(self, ISBN_list, test_dict): #come sopra, inserisco test_dict per fare i test
        self.list_of_books=ISBN_list
        self.ISBN_dictionary=self.create_books_structure(self.list_of_books)
        
      
    def test_create_dictionary(self, ISBN, test_dict):
        book_dict=dict()
        book_dict['Titolo']=(Book(ISBN, test_dict).test_get_title(ISBN, test_dict))[0]
        book_dict['Categoria']=(Book(ISBN, test_dict).test_get_title(ISBN, test_dict))[1]
        book_dict.update(Book(ISBN, test_dict).test_get_book(book_dict['Titolo'], test_dict))
        return book_dict
    
    def create_dictionary(self, ISBN):
        book_dict=dict()
        book_dict['Titolo']=(Book(ISBN).get_title(ISBN))[0]
        book_dict['Categoria']=(Book(ISBN).get_title(ISBN))[1]
        book_dict.update(Book(ISBN).get_book(book_dict['Titolo']))
        return book_dict
        
    def create_books_structure(self, ISBN_list):
        dictionary_of_books=dict()
        for i in range(len(ISBN_list)):
            dictionary_of_books[f'{ISBN_list[i]}']=self.test_create_dictionary(ISBN_list[i], test_dict)  #uso test_create_dictionary al posto della funzione originale per il test
        return dictionary_of_books
                
    def insert_book(self, ISBN):
        self.ISBN_dictionary[f'{ISBN}']=self.test_create_dictionary(ISBN, test_dict) #uso funzione di test al posto dell'originale
        return (self.ISBN_dictionary)
        
    def remove_book(self, ISBN):
        (self.ISBN_dictionary).pop(f'{ISBN}')
        return (self.ISBN_dictionary)
            
    def get_library_value(self):
        net_price=0
        whole_price=0
        for book in self.ISBN_dictionary:
            net_price+=Book(int(book)).get_net_price()
            whole_price+=Book(int(book)).get_whole_price()
        return net_price, whole_price
    
    def test_get_library_value(self): #funzione di test
        net_price=0
        whole_price=0
        for book in self.ISBN_dictionary:
            net_price+=Book(book, test_dict).get_net_price()
            whole_price+=Book(book, test_dict).get_whole_price()
        return net_price, whole_price
        
    def get_total_pages(self):
        total_pages=0
        for book in self.ISBN_dictionary:
            total_pages+=Book(book).get_pages()
        return total_pages
    
    def test_get_total_pages(self): #funzione di test
        total_pages=0
        for book in self.ISBN_dictionary:
            total_pages+=Book(book, test_dict).get_pages()
        return total_pages
        
    def get_categories_number(self):
        list_of_categories=[]
        for book in self.ISBN_dictionary:
            if (self.ISBN_dictionary[book])['Categoria'] not in list_of_categories:
                list_of_categories.append( (self.ISBN_dictionary[book])['Categoria'])
        return len(list_of_categories)
        
        
    def aux_minimum_path(self, start_ISBN, end_ISBN, shortest_path):
        if Book(start_ISBN).is_near(end_ISBN):
            shortest_path.append(end_ISBN)
            return 1, shortest_path
        shortest=len(self.list_of_books)+1
        shortest_ISBN=start_ISBN
        for ISBN in self.list_of_books:
            if ISBN not in shortest_path and Book(start_ISBN).is_near(ISBN):
                if (self.aux_minimum_path(ISBN, end_ISBN, shortest_path))[0]<shortest:
                    shortest=(self.aux_minimum_path(ISBN, end_ISBN, shortest_path))[0]
                    shortest_ISBN=ISBN
        if shortest==len(self.list_of_books)+1:
            print("shortest path couldn't be found because there are no books next to ", start_ISBN)
        shortest_path.append(shortest_ISBN)
        return (1+shortest), shortest_path
    
    def test_aux_minimum_path(self, start_ISBN, end_ISBN, shortest_path, test_dict): #funzione di test
        if Book(start_ISBN, test_dict).test_is_near(end_ISBN, test_dict):
            shortest_path.append(end_ISBN)
            return 1, shortest_path
        shortest=len(self.list_of_books)+1
        shortest_ISBN=start_ISBN
        for ISBN in self.list_of_books:
            if ISBN not in shortest_path and Book(start_ISBN, test_dict).test_is_near(ISBN, test_dict):
                if (self.test_aux_minimum_path(ISBN, end_ISBN, shortest_path, test_dict))[0]<shortest:
                    shortest=(self.test_aux_minimum_path(ISBN, end_ISBN, shortest_path, test_dict))[0]
                    shortest_ISBN=ISBN
        if shortest==len(self.list_of_books)+1:
            print("shortest path couldn't be found because there are no books next to ", start_ISBN)
        shortest_path.append(shortest_ISBN)
        return (1+shortest), shortest_path
        
    def minimum_path(self, start_ISBN, end_ISBN, test_dict):
        shortest_path=[]
        minimum_lenght, shortest_path=self.test_aux_minimum_path(start_ISBN, end_ISBN, shortest_path, test_dict) #uso funzione di test
        shortest_path.append(start_ISBN)
        shortest_path.reverse()
        return minimum_lenght, shortest_path
        
    def aux_minimum_cost_path(self, start_ISBN, end_ISBN, cheapest_path):
        if Book(start_ISBN).is_near(end_ISBN):
            cheapest_path.append(end_ISBN)
            return Book(start_ISBN).get_net_price()+Book(end_ISBN).get_net_price(), cheapest
        cheapest=self.get_library_value()
        cheapest_ISBN=start_ISBN
        for ISBN in self.list_of_books:
            if ISBN not in cheapest_path and Book(start_ISBN).is_near(ISBN):
                if (self.aux_minimum_cost_path(ISBN, end_ISBN, list_of_path))[0]<cheapest:
                    cheapest=(self.aux_minimum_cost_path(ISBN, end_ISBN, cheapest_path))[0]
                    cheapest_ISBN=ISBN
        if cheapest==self.get_library_value():
            print("cheapest path couldn't be found because there are no books next to ", start_ISBN)
        cheapest_path.append(shortest_ISBN)
        return (Book(start_ISBN).get_net_value()+cheapest), cheapest_path
        
    def minimum_cost_path(self, start_ISBN, end_ISBN):
        cheapest_path=[]
        minimum_lenght, cheapest_path=self.aux_minimum_cost_path(start_ISBN, end_ISBN, cheapest_path)
        cheapest_path.append(start_ISBN)
        cheapest_path.reverse()
        return minimum_lenght, cheapest_path
        
ISBN_list=[]
for cat in test_dict:
    for el in (test_dict[cat])[1]:
        ISBN_list.append(el)
print(ISBN_list)
print(My_Library(ISBN_list[70:90], test_dict).ISBN_dictionary)
print(My_Library(ISBN_list, test_dict).test_get_library_value())
print(My_Library(ISBN_list, test_dict).test_get_total_pages())
print(My_Library(ISBN_list, test_dict).get_categories_number())
print(len(ISBN_list))

new_list=ISBN_list[60:130]





print(My_Library(new_list, test_dict).minimum_path('9788845935053', '9788845932441', test_dict)) 


['9788845935053', '9788845934513', '9788845934223', '9788845933646', '9788845932878', '9788845932182', '9788845931925', '9788845931659', '9788845931505', '9788845929625', '9788845929250', '9788845927850', '9788845927669', '9788845927034', '9788845926433', '9788845926044', '9788845924637', '9788845924026', '9788845921537', '9788845920509', '9788845921193', '9788845918704', '9788845917226', '9788845916601', '9788845915512', '9788845911958', '9788845908736', '9788845906893', '9788845907197', '9788845933974', '9788845933301', '9788845932625', '9788845932540', '9788845930096', '9788845929960', '9788845929342', '9788845929328', '9788845928093', '9788845926969', '9788845926716', '9788845926150', '9788845925351', '9788845924934', '9788845922527', '9788845917974', '9788845916250', '9788845915680', '9788845913969', '9788845912825', '9788845912221', '9788845911811', '9788845909771', '9788845907531', '9788845908026', '9788845933875', '9788845933868', '9788845933851', '9788845933844', '978884593383